In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import shutil
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report

2023-10-24 16:53:49.232618: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-24 16:53:49.502469: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-24 16:53:49.504327: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-24 16:53:50.777728: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
batch_size = 32
img_height = 224
img_width = 224

In [3]:
def dynamic_data_split(data_dir, train_size=0.7, val_size=0.15, test_size=0.15):
    split_main_dir = os.path.join(data_dir, 'splits')
    if not os.path.exists(split_main_dir):
        os.makedirs(split_main_dir)
    
    train_dir = os.path.join(split_main_dir, 'train')
    val_dir = os.path.join(split_main_dir, 'val')
    test_dir = os.path.join(split_main_dir, 'test')


    for directory in [train_dir, val_dir, test_dir]:
        if not os.path.exists(directory):
            os.makedirs(directory)


    for class_dir in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_dir)
        if os.path.isdir(class_path) and class_path != split_main_dir:

            files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
            
        
            if files:
                train_files, temp_files = train_test_split(files, train_size=train_size, random_state=123)
                val_files, test_files = train_test_split(temp_files, train_size=(val_size/(val_size + test_size)), random_state=123)
                
            
                def copy_files(files, directory):
                    class_directory = os.path.join(directory, class_dir)
                    if not os.path.exists(class_directory):
                        os.makedirs(class_directory)
                    for f in files:
                        shutil.copy2(os.path.join(class_path, f), os.path.join(class_directory, f))

            
                copy_files(train_files, train_dir)
                copy_files(val_files, val_dir)
                copy_files(test_files, test_dir)
            else:
                print(f"No samples found in directory {class_path}, skipping split.")

In [4]:
data_dir = '/home/maria/Desktop/Master degree/2 course/computer vision/hmw2/Dataset/splits'
dynamic_data_split(data_dir, train_size=0.7, val_size=0.15, test_size=0.15)

No samples found in directory /home/maria/Desktop/Master degree/2 course/computer vision/hmw2/Dataset/splits/train, skipping split.
No samples found in directory /home/maria/Desktop/Master degree/2 course/computer vision/hmw2/Dataset/splits/val, skipping split.
No samples found in directory /home/maria/Desktop/Master degree/2 course/computer vision/hmw2/Dataset/splits/test, skipping split.


In [5]:
BATCH_SIZE = 32
img_height = 224
img_width = 224
epochs = 1
LR = 0.0001
num_classes = 3

In [6]:
train_dir = data_dir + '/train'
val_dir = data_dir + '/val'
test_dir = data_dir + '/test'

In [7]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


Found 1959 images belonging to 3 classes.
Found 420 images belonging to 3 classes.


In [8]:
base_model = ResNet50(weights='imagenet', include_top=False)

In [9]:
model = Sequential([
  base_model,
  GlobalAveragePooling2D(),
  Dense(num_classes, activation='softmax')
])

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

2023-10-24 16:53:55.876762: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


62/62 [==============================] - ETA: 0s - loss: 0.4917 - accuracy: 0.8806

2023-10-24 17:04:14.088753: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


62/62 [==============================] - 648s 10s/step - loss: 0.4917 - accuracy: 0.8806 - val_loss: 633.6492 - val_accuracy: 0.6214


In [12]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy : {accuracy}')

Found 422 images belonging to 3 classes.


2023-10-24 17:04:44.112966: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


14/14 [==============================] - 31s 2s/step - loss: 604.2007 - accuracy: 0.6374
Test accuracy : 0.6374407410621643


In [13]:
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())
conf_matrix = confusion_matrix(true_classes, predicted_classes)

2023-10-24 17:05:14.883304: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


14/14 [==============================] - 31s 2s/step


In [14]:
conf_matrix

array([[78, 50,  0],
       [85, 62,  0],
       [97, 50,  0]])